Working file for Goup Project 

Data
-   https://www.fao.org/faostat/en/#data/QCL
-   https://data.worldbank.org/
-   

Importing all libarys 

In [16]:
import pandas as pd
import numpy as np
import zipfile
import seaborn as sns # For plotting
import matplotlib.pyplot as plt # For showing plots
import scipy.stats as sts
import scikit_posthocs as sp
import statsmodels.stats as stm
from statsmodels.graphics.gofplots import qqplot


1. Dataset

Import

In [17]:
df = pd.read_csv("data/Production_Crops_Livestock_E_All_Data.csv", encoding='latin-1')

Cleaning

In [18]:
df2 = df.dropna(how = 'any') # removing all missing data 
df2 = df2[df2["Area Code"]<1000] # removing the summerised Areas (Africa as a hole etc.)
df2 = df2.drop(["Area Code", "Area Code (M49)","Item Code (CPC)" ,"Item Code", "Element Code"], axis=1) # removing not used collums
df2 = df2.drop(df2.columns[df2.columns.str.endswith('F')], axis=1) # removing all collums wich say how the data was collectet. Big loss of information!! 
df2 = pd.melt(df2, id_vars=['Area', "Item", "Element", "Unit"], var_name='Year', value_name='Value') # reshaping the df to create a single year collum 
df2['Year'] = df2['Year'].str[1:] # removin the letter of befor the years
df2['Year'] = df2['Year'].astype(int)
df2 = df2.rename(index=str, columns={"Value": "yield_amound"})
df2

,Area,Item,Element,Unit,Year,yield_amound
0,Afghanistan,Apples,Area harvested,ha,1961,2220.00
1,Afghanistan,Apples,Yield,hg/ha,1961,68018.00
2,Afghanistan,Apples,Production,tonnes,1961,15100.00
3,Afghanistan,Apricots,Area harvested,ha,1961,4820.00
4,Afghanistan,Apricots,Yield,hg/ha,1961,66390.00
...,...,...,...,...,...,...
1772777,Zimbabwe,Sugar Crops Primary,Yield,hg/ha,2021,734043.00
1772778,Zimbabwe,Sugar Crops Primary,Production,tonnes,2021,3450000.00
1772779,Zimbabwe,Vegetables Primary,Area harvested,ha,2021,35148.00
1772780,Zimbabwe,Vegetables Primary,Yield,hg/ha,2021,66770.00


2. Dataset 

Import

In [19]:
dr = pd.read_csv("data/FAOSTAT_data_en_4-18-2023.csv", encoding='latin-1')

Cleaning

In [20]:
dr2 = dr.dropna(how = 'any') # removing all missing data 
dr2 = dr2.drop(["ï»¿Domain Code", "Area Code (M49)", "Item Code", "Element Code", "Year Code", "Flag", "Flag Description"], axis=1) # removing not used collums
dr2 = dr2.rename(index=str, columns={"Value": "pesticide_amound"})

3. Dataset 

Import

In [21]:
di = pd.read_csv("data/temp.csv", encoding='latin-1')

Cleaning

In [22]:
di2 = di.dropna(how = 'any') # removing all missing data 
di2 = di2.rename(index=str, columns={"year": "Year"}) # renaming collums 
di2 = di2.rename(index=str, columns={"country": "Area"}) #renaming collums 
di2

,Year,Area,avg_temp
0,1849,CÃ´te D'Ivoire,25.58
1,1850,CÃ´te D'Ivoire,25.52
2,1851,CÃ´te D'Ivoire,25.67
7,1856,CÃ´te D'Ivoire,26.28
8,1857,CÃ´te D'Ivoire,25.17
...,...,...,...
71306,2009,Mexico,21.76
71307,2010,Mexico,20.90
71308,2011,Mexico,21.55
71309,2012,Mexico,21.52


Merge the Datasets 

In [23]:
merged = pd.merge(df2, dr2, on= ["Area", "Year"])
merged = pd.merge(di2, merged, on= ["Area", "Year"])
merged

,Year,Area,avg_temp,Item_x,Element_x,Unit_x,yield_amound,Domain,Element_y,Item_y,Unit_y,pesticide_amound
0,1990,Ghana,26.73,Asses,Stocks,Head,10398.00,Pesticides Use,Agricultural Use,Pesticides (total),tonnes,65.8
1,1990,Ghana,26.73,Asses,Stocks,Head,10398.00,Pesticides Use,Agricultural Use,Insecticides,tonnes,32.4
2,1990,Ghana,26.73,Asses,Stocks,Head,10398.00,Pesticides Use,Agricultural Use,Herbicides,tonnes,8.0
3,1990,Ghana,26.73,Asses,Stocks,Head,10398.00,Pesticides Use,Agricultural Use,Fungicides and Bactericides,tonnes,25.0
4,1990,Ghana,26.73,Asses,Stocks,Head,10398.00,Pesticides Use,Agricultural Use,Fungicides â Seed treatments,tonnes,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
15893010,2013,Namibia,20.73,Vegetables Primary,Production,tonnes,60587.49,Pesticides Use,Agricultural Use,Insecticides,tonnes,9.0
15893011,2013,Namibia,20.73,Vegetables Primary,Production,tonnes,60587.49,Pesticides Use,Agricultural Use,Herbicides,tonnes,17.0
15893012,2013,Namibia,20.73,Vegetables Primary,Production,tonnes,60587.49,Pesticides Use,Agricultural Use,Fungicides and Bactericides,tonnes,18.0
15893013,2013,Namibia,20.73,Vegetables Primary,Production,tonnes,60587.49,Pesticides Use,Agricultural Use,Plant Growth Regulators,tonnes,4.0


In [24]:
merged.describe()

,Year,avg_temp,yield_amound,pesticide_amound
count,1.589302e+07,1.589302e+07,1.589302e+07,1.589302e+07
mean,2.001206e+03,1.962269e+01,6.218698e+06,1.414622e+04
std,6.787649e+00,6.921072e+00,3.210777e+07,4.090813e+04
min,1.990000e+03,-3.370000e+00,0.000000e+00,0.000000e+00
25%,1.995000e+03,1.523000e+01,1.342900e+04,3.657000e+01
50%,2.001000e+03,2.104000e+01,1.096900e+05,7.427100e+02
75%,2.007000e+03,2.592000e+01,8.300000e+05,6.763400e+03
max,2.013000e+03,3.042000e+01,7.680904e+08,3.677780e+05
